In [2]:
import sys
import simulator as sim
import numpy as np

SEC = 1000000
MIN = 60 * SEC
DOLLAR = 1000000


###############################################################################
# Store daily book data in a .txt file
###############################################################################


class StoreDayData(object):
    
    def __init__(self, session, date, ticker, 
                 num_levels = 5,
                 start_time = sim.string_to_micro("09:00"), 
                 end_time = sim.string_to_micro("17:00")):
        self.session = session
        self.date = date
        self.ticker = ticker[0]
        self.start_time = start_time
        self.end_time = end_time
    
        self.num = (self.end_time - self.start_time) / MIN
        self.num_levels = num_levels
        self.time_index = 0
        
        # book_data is a ndarray with columns
        #     0: time
        #     1: average price (to obtain the output signal to feed the SVM)
        #   2-6: queue size of ask levels 1-5
        #  7-11: queue size of bid levels 1-5
        self.book_data = np.empty(shape = [self.num, 2 * self.num_levels + 2])
        
        self.session.subscribe_ticker_all_feeds(self.ticker)
        for i in range(self.num):
            self.session.add_timer(self.start_time + i * MIN, self.timer_getdata_callback)    
        
        
    def timer_getdata_callback(self, time):
        book = self.session.get_book_levels(self.ticker, nlevels = self.num_levels)
        bids = book["bids"]
        asks = book["asks"]
        
        self.book_data[self.time_index, 0] = time
        best_bid, best_ask = self.session.get_inside_market(self.ticker)
        self.book_data[self.time_index, 1] = (bids[0]["price"] + asks[0]["price"]) / 2.0 / DOLLAR
        
        for i in range(self.num_levels):
            self.book_data[self.time_index, i + 2] = asks[i]["size"]
            self.book_data[self.time_index, i + 7] = bids[i]["size"]
            
        self.time_index += 1
            
            
    def end(self):
        time = self.session.current_time()
        # print "DONDONDON: it's now {0}".format(sim.micro_to_time(time))
        filename = "{0}_{1}_bookdata.txt".format(self.ticker, self.date)
        np.savetxt(filename, self.book_data, newline = "\n")
        print "Saved to {0}".format(filename)
        return


###############################################################################
# Read from book data file and convert it to feedable SVM data
###############################################################################
    
    
def treat_data(ticker, date, threshold = 0.01):
    raw_filename = "{0}_{1}_bookdata.txt".format(ticker, date) 
    raw_data = np.loadtxt(raw_filename)
    num_data = raw_data.shape[0]
    num_lvls = (raw_data.shape[1] - 2) / 2
    buildup = [1, 2, 3, 5, 10, 15]
    
    # sig_data is the output signal for the SVM, as a (n, 1) ndarray
    sig_data = np.empty(shape = [num_data - 15, 1])
    # obp_data is the SVM input as a (n, 6 * num_lvls) ndarray where
    # OBP(t, l) = obp_data[ , t*num_lvls + + l]
    obp_data = np.empty(shape = [num_data - 15, 6 * num_lvls])
    
    for i in range(15, num_data):
        price_change = (raw_data[i, 1] - raw_data[i - 1, 1]) / raw_data[i - 1, 1]
        if price_change < -threshold:
            sig_data[i-15, 0] = -1
        elif price_change > threshold:
            sig_data[i-15, 0] = 1
        else:
            sig_data[i-15, 0] = 0
            
        for t in range(len(buildup)):
            for j in range(num_lvls):
                obp_data[i-15, t*num_lvls + j] = np.sum(raw_data[(i-buildup[t]):(i+1), 7:(j+8)]) / np.sum(raw_data[(i-buildup[t]):(i+1), 2:(j+3)])
                            
    obp_filename = "{0}_{1}_obp.txt".format(ticker, date)
    sig_filename = "{0}_{1}_sig.txt".format(ticker, date)
    np.savetxt(obp_filename, obp_data, newline = "\n")
    np.savetxt(sig_filename, sig_data, newline = "\n")
    print "Saved to {0}".format(obp_filename)
    
    
###############################################################################
# Default algorithm with best bid best ask
###############################################################################


class TestAlgo:
    
    def __init__(self, session, date, ticker, start_time, end_time):
        # Save session information
        self.session = session
        self.date = date
        self.tickers = ticker
        self.start_time = start_time
        self.end_time = end_time
        self.order_size = 1
        self.mu = 0
        self.ORDER_WAIT = 30
        
        # outstanding_orders is a dictionary, tickers are the keys and values are 
        # {"A": [ask_order_id, time_accepted], "B": [bid_order_id, time_accepted]}
        # if there are outstanding orders
        self.outstanding_orders = {}
        
        # signal is a dictionary : {symbol : symbol_signal}
        self.signal = {}
        
        # inventory is another dictionary: {symbol : inventory_size for symbol}
        self.inventory = {}
        self.cash = 0
        
        # Suscribe to tickers and set first timer
        # no trading on the 1st minute: use to compute signal
        num = (end_time - start_time) / MIN
        for i in range(num):
            self.session.add_timer(self.start_time + (i + 1) * MIN, self.timer_OBS_callback)
        
        self.session.subscribe_ticker_all_feeds(self.tickers[0])
        self.outstanding_orders[self.tickers[0]] = {}
        self.signal[self.tickers[0]] = self.get_signal(self.tickers[0])
        self.inventory[self.tickers[0]] = 0
        self.session.subscribe_event(self.tickers[0], sim.ORDER_ACCEPTED, self.order_ack_callback)
        self.session.subscribe_event(self.tickers[0], sim.ORDER_EXECUTED, self.order_exe_callback)
        self.session.subscribe_event(self.tickers[0], sim.ORDER_CANCELED, self.cancelled_callback)
        self.session.subscribe_event(self.tickers[0], sim.ORDER_REJECTED, self.rejected_callback)
        
        
    def timer_OBS_callback(self, time):
        #print "OBS callback: {0}, {1}".format(sim.micro_to_time(time), sim.micro_to_time(self.end_time))
        if (time >= self.end_time):
            self.end()
            return
        for sym_it in self.signal:
            curr_sig = self.get_signal(sym_it)
            if (curr_sig != self.signal[sym_it]):
                cancel_orders(sym_it)
                stop_loss(sym_it)
                self.signal[sym_it] = curr_sig
            self.post_orders(sym_it)
            
            
    def post_orders(self, ticker):
        now = self.session.current_time()
        orders = self.session.get_orders_by_ticker(ticker)
        if not orders:
            bid, ask = self.session.get_inside_market(ticker)
            b_price = bid['price'] # = bestBidPrice()
            a_price = ask['price']
            self.session.add_order(ticker, sim.BUY, self.order_size, b_price + self.signal[ticker] * self.mu, exchange=sim.EXCH_INET)
            self.session.add_order(ticker, sim.SELL, self.order_size, a_price + self.signal[ticker] * self.mu, exchange=sim.EXCH_INET) 
            print "Posted orders"
               
    
    def cancel_orders(self, ticker):
        now = self.session.current_time()
        #print "cancel order call back: it's now {0}".format(sim.micro_to_time(now))
        ord_info = self.outstanding_orders[ticker]
        if len(ord_info) == 0:
            print "no order to cancel"
            return
        else:
            for side in ord_info:
                sim.cancel_order(ord_info[side][0])
                print "cancelled order {0}".format(ord_info[side][0])
        
        
    def stop_loss(self, ticker):
        print "stop loss for ticker {0}".format(ticker)
        inv = self.inventory[ticker]
        if (inv == 0):
            return
        elif (inv > 0):
            # not the right way to send market order
            self.session.add_order(ticker, sim.SELL, inv, 0, exchange=sim.EXCH_INET)
            print "selling inventory"
        else:
            # not the right way to send market order
            self.session.add_order(ticker, sim.BUY, -inv, 999 * 1000000, exchange=sim.EXCH_INET)
            print "buying inventory"
        
        
    def timer_order_manage_callback(self, time):
        now = self.session.current_time()
        #print "order manage back: it's now {0}".format(sim.micro_to_time(now))
        for sym_it in self.outstanding_orders:
            ord_info = self.outstanding_orders[sym_it]
            if len(ord_info) == 0:
                continue
            else:
                for side in ord_info:
                    cancel_time = ord_info[side][1]
                    if (now < cancel_time):
                        #this may add duplicate timer event for different symbols
                        self.session.add_timer(cancel_time, self.timer_order_manage_callback)
                    else:
                        sim.cancel_order(ord_info[side][0])
                        print "Cancelled order {0}".format(ord_info[side][0])
         
        
    def rejected_callback(self, ticker, event_params):
        print "rejected!!!!"
        print event_params['rejected_orders']
    
    
    def order_ack_callback(self, ticker, event_params):
        time = self.session.current_time()
        #print "Ack Call back: it's now {0}".format(sim.micro_to_time(time))
        for order in event_params['accepted_orders']:
            ord_id = order['order_id']
            sym = order['ticker']
            side = order['side']
            t = order['time_accepted']
            self.outstanding_orders[sym][side] = [ord_id, t]
            #print "Order accepted. order id is {0}, symbol is {1}, side is {2}, time is {3}".format(ord_id, sym, side, sim.micro_to_time(time))
        
        self.session.add_timer(t + self.ORDER_WAIT * SEC, self.timer_order_manage_callback)    
        
        
    def order_exe_callback(self, ticker, event_params):
        time = self.session.current_time()
        #print "Execution call_back: it's now {0}".format(sim.micro_to_time(time))
        for exe in event_params['executed_orders']:
            order = exe['order']
            ord_id = order['order_id']
            sym = order['ticker']
            side = order['side']
            t = order['time_placed']
            self.outstanding_orders[sym].pop(side)
            if (side == "B"):
                self.inventory[sym] += exe['quantity_executed'] 
                self.cash -= exe['quantity_executed'] * exe['price_executed']
                if "S" in self.outstanding_orders[sym]:
                    self.outstanding_orders[sym]["S"][1] = time + self.ORDER_WAIT * SEC
                    print "extending timer for sell order"
            else:
                self.inventory[sym] -= exe['quantity_executed'] 
                self.cash += exe['quantity_executed'] * exe['price_executed']
                if "B" in self.outstanding_orders[sym]:
                    self.outstanding_orders[sym]["B"][1] = time + self.ORDER_WAIT * SEC
                    print "extending timer for buy order"
            #print "Order Executed. order id is {0}, symbol is {1}, side is {2}, time is {3}, dollar amount is {4}".format(ord_id, sym, side, sim.micro_to_time(time), exe['quantity_executed'] * exe['price_executed'])
    
    
    def cancelled_callback(self, ticker, event_params):
        time = self.session.current_time()
        #print "Cancelled call back: it's now {0}".format(sim.micro_to_time(time))
        for order in event_params['canceled_orders']:
            ord_id = order['order_id']
            sym = order['ticker']
            side = order['side']
            t = order['time_placed']
            self.outstanding_orders[sym].pop(side)
            #print "Order Cancelled. order id is {0}, symbol is {1}, side is {2}, time is {3}".format(ord_id, sym, side, sim.micro_to_time(time))
        
        
    #returns -1, 0, or 1 corresponding to markets going up, even, or down
    def get_signal(self, ticker):
        return 0
    
    
    def calculate_pnl(self, time):
        print "PnL is {0}".format(self.cash / float(1000000))
    
    
    def end(self):
        time = self.session.current_time()
        print "DONDONDON: it's now {0}".format(sim.micro_to_time(time))
        print "Done"
        for sym_it in self.outstanding_orders:
            self.cancel_orders(sym_it)
        print "cash: {0}".format(self.cash)
        for sym_it in self.tickers:
            print "inventory: {0}".format(self.inventory[sym_it])    
            self.stop_loss(sym_it)
        
        self.session.add_timer(time + MIN, self.calculate_pnl)
        return 
    

###############################################################################
# Main
###############################################################################


date = "20150121"
training_range = range(int(date) - 10, int(date))
symbols = ["IVV"]
start_time = sim.string_to_micro("9:30")
end_time = sim.string_to_micro("12:30")

#for training_date in training_range:
#    simul_storedata = sim.Simulator(StoreDayData)
#    simul_storedata.run(str(training_date), symbols, num_levels = 5, start_time = start_time, end_time = end_time)

threshold = 0.0005
treat_data("IVV", "20150105", threshold)
    
#simul_trading = sim.Simulator(TestAlgo)
#simul_trading.run(date, symbols, use_om=True, start_time=start_time, end_time=end_time)

ImportError: No module named simulator